In [1]:
import numpy as np
import sympy as sp

In [2]:
def criar_vetor_de_simbolos(tamanho):
    """
    Cria um vetor (lista) de símbolos SymPy com o tamanho especificado.

    Parameters:
    tamanho (int): O número de símbolos a serem criados.

    Returns:
    list: Lista contendo os símbolos criados.
    """
    # Cria os símbolos usando sp.symbols
    return sp.symbols(f'x1:{tamanho+1}')

In [3]:
sp.Matrix(np.array_split(criar_vetor_de_simbolos(4), 2))

Matrix([
[x1, x2],
[x3, x4]])

In [4]:
# Função para calcular o produto Kronecker considerando binário exclusivo
def kronecker_binary(vectors):
    result = np.array([1])  # Inicia com o elemento neutro da multiplicação
    for v in vectors:
        result = np.kron(result, v)
    return result

In [5]:
kronecker_binary([[1,0],[0,1]])

array([0, 1, 0, 0])

In [ ]:
bits = 2
variaveis = sp.Matrix(kronecker_binary(np.array_split(criar_vetor_de_simbolos(bits*2), bits)))
variaveis

Matrix([
[x1*x3],
[x1*x4],
[x2*x3],
[x2*x4]])

In [ ]:
def decimal_para_binario_vetor(numero_decimal):
    """
    Converte um número decimal para um vetor binário de 1 bits usando produtos de Kronecker.

    Args:
        numero_decimal (int): úmero decimal a ser convertido.

    Returns:
        np.ndarray: Vetor binário resultante.
    """
    # Converter o número para uma string binária de 1 bits
    numero_binario = format(numero_decimal, '01b')
    
    # Mapeamento simples para cada bit
    mapeamento = {'0': [1, 0], '1': [0, 1]}
    
    # Inicializa o vetor com [1]
    vetor = np.array([1])
    
    # Aplica o produto de Kronecker para cada bit
    for bit in numero_binario:
        vetor = np.kron(vetor, mapeamento[bit])
    
    return vetor

In [32]:
numeros_decimais = [0,1,1,0]
vetores_binarios = []
tamanho_bits = 1

for numero in numeros_decimais:
    try:
        vetor = decimal_para_binario_vetor(numero)
        vetores_binarios.append(vetor)
    except (TypeError, ValueError) as e:
        print(f"Erro ao converter o número {numero}: {e}")

vetores_binarios = np.array(vetores_binarios).T
sp.Matrix(vetores_binarios)

Matrix([
[1, 0, 0, 1],
[0, 1, 1, 0]])

In [11]:
regra = sp.Matrix(np.dot(vetores_binarios, kronecker_binary(np.array_split(criar_vetor_de_simbolos(4), 2)))) # expressões simbólicas
regra

Matrix([
[x1*x3 + x2*x4],
[x1*x4 + x2*x3]])

In [14]:
#  Lista para armazenar as funções
funcoes = []

# Variáveis que as funções irão receber
variaveis = criar_vetor_de_simbolos(4)

# Iterar sobre cada elemento do vetor simbólico e criar uma função
for expr in regra:
    func = sp.lambdify(variaveis, expr)
    funcoes.append(func)

funcoes

[<function _lambdifygenerated(x1, x2, x3, x4)>,
 <function _lambdifygenerated(x1, x2, x3, x4)>]

In [15]:
from sympy.utilities.lambdify import lambdastr

# Variáveis que as funções irão receber
variaveis = criar_vetor_de_simbolos(4)

for expr in regra:
    print(lambdastr(variaveis, expr))

lambda x1,x2,x3,x4: (x1*x3 + x2*x4)
lambda x1,x2,x3,x4: (x1*x4 + x2*x3)


In [ ]:
valores = np.array([decimal_para_binario_vetor(0,1),decimal_para_binario_vetor(0,1)]).flatten()
print(valores)

# Calcular os resultados
[f(*valores) for f in funcoes]

[1 0 1 0]


[1, 0, 0, 0]

In [175]:
sp.Matrix(np.dot(vetores_binarios, [0,1,0,0]))

Matrix([
[0],
[0],
[1],
[0]])

In [18]:
import itertools

# Usando produto cartesiano
combinacoes = list(itertools.product([0, 1], repeat=2))

for comb in combinacoes:
	# Converte cada bit da combinação para um vetor binário e achata em uma única dimensão
	valores = np.array([decimal_para_binario_vetor(bit, 1) for bit in comb]).flatten()
	print(valores,[f(*valores) for f in funcoes])

[1 0 1 0] [1, 0]
[1 0 0 1] [0, 1]
[0 1 1 0] [0, 1]
[0 1 0 1] [1, 0]
